**Notebook for Problem 3**

In [26]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Use Pandas to read tables from webpage

In [27]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)
df_Toronto = dfs[0]

df_Toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Remove non-assigned boroughs

In [31]:
df_AssignedBorough= df_Toronto[df_Toronto.Borough != 'Not assigned']
df_AssignedBorough.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [33]:
df_AssignedBorough_gb = df_AssignedBorough.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join)
df_AssignedBorough_gb.head()

Postcode  Borough    
M1B       Scarborough                           Rouge,Malvern
M1C       Scarborough    Highland Creek,Rouge Hill,Port Union
M1E       Scarborough         Guildwood,Morningside,West Hill
M1G       Scarborough                                  Woburn
M1H       Scarborough                               Cedarbrae
Name: Neighbourhood, dtype: object

In [34]:
df_concat = df_AssignedBorough_gb.to_frame()

df_concat.head()

,,Neighbourhood
Postcode,Borough,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [35]:
print(len(df_concat.index.levels))
df_concat=df_concat.reset_index(level=[0,1])
df_concat

2


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [36]:
print(df_concat.loc[df_concat['Neighbourhood'] == 'Not assigned'])
df_concat['Neighbourhood'] = np.where(df_concat['Neighbourhood'] == 'Not assigned', df_concat['Borough'], df_concat['Neighbourhood'])

   Postcode       Borough Neighbourhood
85      M7A  Queen's Park  Not assigned


In [37]:
print(df_concat.loc[df_concat['Neighbourhood'] == 'Not assigned'])


Empty DataFrame
Columns: [Postcode, Borough, Neighbourhood]
Index: []


In [38]:
df_concat.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [39]:
df_concat.shape

(103, 3)

In [40]:
df_concat = df_concat.groupby(['Postcode','Borough'])['Neighbourhood'].agg(','.join).reset_index()

In [41]:
df_concat.loc[df_concat['Neighbourhood'] == 'Not assigned','Neighbourhood'] = df_concat.loc[df_concat['Neighbourhood'] == 'Not assigned'].Borough

In [42]:
df_cord = pd.read_csv('Geospatial_Coordinates.csv')
df_cord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [43]:
df_concat_cord = df_concat.merge(df_cord,how='left',left_on='Postcode',right_on='Postal Code')
df_concat_cord.drop('Postal Code',inplace=True,axis=1)
df_concat_cord.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [44]:
df_concat_cord.shape

(103, 5)

**Neighbourhood Analysis**

In [45]:
import folium
map_toronto = folium.Map(location=[43.653908, -79.384293], zoom_start=10)

In [49]:
# add markers to map
for lat, lng, borough, neighborhood in zip(df_concat_cord['Latitude'], df_concat_cord['Longitude'], df_concat_cord['Borough'], df_concat_cord['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto